# BASTA (a.k.a Scattergories)
## Game Description
**Basta (a.k.a. Scattergories | Tutti Frutti)** is a creative-thinking category-based game in which the objective is to score points by uniquely naming objects within a set of categories (in this case: _Name, Last Name, Country, Fruit, Animal, Color, and Thing_), given an initial letter, within a time limit. For the purpose of this project, the game is set in spanish, to measure in a fun way the player's mental agility and knowledge of this language. 

### The Beginning
To build this game, we shall start by importing all the resources (a.k.a _libraries_) that we're going employ in the code ...

In [23]:

# to avoid any problems, it is best to import everything outside of all the functions that are soon to be defined ...

from collections import Counter
from unicodedata import normalize
import numpy as np
import os
import pandas as pd
import pyautogui
import random
import re
import string 


Now the coding begins, for starters we're going to define a function that returns a random letter from the alphabet ... 

_Remember from the game description that one of the criteria for scoring points, is that the player's answers have to start with a respective initial letter._

In [40]:

def alpha():
    abecedario = list(string.ascii_lowercase)
    # the string library returns the alphabet in a sole string 'abcdef...', by using the list function we create a list with all            letters of the alphabet ['a', 'b', 'c', ...]
    letra = random.choice(abecedario)
    # using random.choice we gather one letter from the list previously defined.
    return letra


As the next step, we're going to define the function that is going to initiate the game by asking the player its answers for each category ...

In [41]:
def play(run, letra, campos): 
    '''
    play() is the responsible function to 'run' the game, as it is going to enable the player to fill their answers.
    '''
    # run = the decision of the player of initiating the game or not.
    # letra = the letter choiced by the function alpha.
    # campos = the categories or questions of the game.
    # all of the above are going to be defined in the game's main function
    respuestas = []
    # this is an empty list, in which the user's inputs are going to be saved.
    if run == "Estoy Listo": # this translates into the player accepting to play
        pyautogui.alert('Todo lo que sepas con la Letra %s' % letra, title='Letra')
        # with pyautogui we're creating a pop-up message box, in which we're warning the player about the letter to take into                   consideration when answering.
        for campo in campos:
            respuestas.append(pyautogui.prompt('%s:' % campo, 'Responde :) la letra es %s' % letra,\
                default='Escribe lo primero que se te ocurra ...',root='',timeout=10000))
            # with pyautogui.prompt we're creating an input as a message box for each category / question of the game (*7)
            # pyautogui.prompt(
            #                   question / text,
            #                   title of the message box, 
            #                   default answer, 
            #                   root, 
            #                   timeout = the time limit in milliseconds that the console is going to wait for an input ) 
        print('Tus Respuestas Cerebrito: ', respuestas, '\n')
    else:
        print('¡Bu!¡Que hueva me das!'.upper(), '\n') # this would be the message if the user is not willing to play
    return respuestas or print('¡Hasta la Próxima!'.upper(), '\n')

By considering the fact that the player is going to be under time pressure, we decided to create a mockup spell checker to correct any typo (_regarding numbers, special characters, and small spelling mistakes_) ...

In [42]:
def clean_ans(answers):
    '''
    clean_ans(list_of_strings) is a function designed to clean the user's inputs regarding symbols, numbers, and
    special characters ... as a plus is going to correct the answers that have more than two letters repeated, and
    it is going to ignore any answer with less than three letters. So this function is going to receive a list of 
    strings to return the same list but with clean and lowercase string.
    '''
    # answers = is the list of answers created from the play() function
    answers = [answer.replace(' ', '') for answer in answers] 
    # since each answer is supposed to be one word, we're deleting any unnecessary spaces ' '.
    answers = [answer.replace('Escribeloprimeroqueseteocurra...', 'NA').replace('Timeout', 'NA') for answer in answers]
    # any invalid answer is going to be replace with 'NA'.
    conv = lambda answer : answer or 'NA'
    respuestas_c_NA = [conv(answer) for answer in answers] 
    # here, we used an express function 'lambda' to create a new list with the valid and NA answers.
    minusculas = ' '.join([answer.lower() for answer in respuestas_c_NA])
    # to change all the answers to lowercase, in order to do so, we converted the list of strings into a sole string.
    print('MINUSCULAS:', minusculas, '\n')
    # this print is for quality control.
    sin_cespeciales = re.sub(r'[^a-zA-Z0-9_\s]|\d', '', minusculas)
    # we're employing regex to replace any number or symbol found in the create string from above.
    sin_acentos = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize("NFD", sin_cespeciales), 0, re.I)
    # again, we're using regex, but this time to delete any tilde.
    s_letras_repetidas = re.sub(r'(.)\1{2,}', r'\1', sin_acentos)
    # and one more time, regex sub is going to run another cleaning, but right now to delete any letter that repeats more than two          times, for example: 'holaaa' -> 'hola'
    palabras = s_letras_repetidas.split()
    # finally, we turn the string back to a list of strings.
    print('PALABRAS:', palabras, '\n')
    # this print is for quality control.
    return palabras

### Dictionary
We downloaded a spanish dictionary for each letter of the alphabet, all of which are found in the directory: _dics_ 
By referencing to the dictionaries, we're going to correct to run a spelling corrector ...

In [43]:
def dictionary(path, palabra): # path is going to be defined in the main function of the game.
    def words(text): return re.findall(r'\w+', text.lower())
    WORDS = Counter(words(open(path).read())) 
    def P(word, N=sum(WORDS.values())): 
        "Probability of `word`."
        return WORDS[word] / N
    def correction(word): 
        "Most probable spelling correction for word."
        return max(candidates(word), key=P)
    def candidates(word): 
        "Generate possible spelling corrections for word."
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    def known(words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)
    def edits1(word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    def edits2(word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    return correction(palabra)


### The Final Parts of the Code ...
The following function was built to check that the answers start with the supposed initial letter ...

In [44]:
def check_ans(answers, letra, questions):
    '''
    This function is going to validate all answers that are going to be displayed
    in the 'Final Card' which are the final results of the game.
    '''
    # answers = list of answers that passed through the clean_ans() function
    # letra = the letter choiced by alpha()
    # questions = list of categories <campos>
    pattern = '^%s[a-z]{2,}' % letra
    # this is the pattern followed to validate all answers.
    print('Regex Patron', pattern, '\n')
    # this print is for quality control purposes.
    respuestas_finales = [find for answer in answers for find in re.findall(pattern, answer) or ['???']]
    # this list comprehension is going to pass validate answers into a final list ...
    # and also any invalid answer is going to be passed as '???'
    while len(respuestas_finales) < len(questions):
        respuestas_finales.append('???')
    # this part of the code is to complete any missing categories that were not filled
    # the main reason behind this was because with pandas any None value is going to be ignored
    # which didn't help us as we're looking to create a dataframe with all the categories as columns 
    # even if there were missing answers or inputs.
    print(respuestas_finales, '\n')
    # this print is for quality control purposes.
    return respuestas_finales

This function is responsible for displaying the overall scores of the player ...

In [45]:
def finalcard(answers, questions, letra):
    '''
    This function receives the checked answers from the function above, 
    to create a dictionary Question : Answer from which a dataframe is returned
    by using PANDAS.
    '''
    dict_quest = {i:questions[i] for i in range(len(questions))} 
    # this is a dictionary created from list comprehension.
    data_frame = pd.DataFrame(np.array(answers).reshape(-1,len(answers))).rename(columns=dict_quest).rename(index=lambda row: letra)
    # by combining numpy and pandas we create a personalized data frame.
    # the columns are the categories of the game.
    # the index is the letter that was defined by alpha()
    return data_frame

The following is a function that prints the rules of the game (_*in spanish_) ...

In [46]:
def reglas():
    print('''¡BASTA!
    
LAS REGLAS DEL JUEGO:

1. Deberás escribir una palabra correspondiente a cada categoría, 
solo tendrás 10 segundos por cada categoría y no cumplir con el tiempo resulta en “timeout”.

2. Cada palabra deberá iniciar con la letra que se escogió al azar.

3. Te ayudaremos con “typos” pero ¡no somos advinos! 

Recomendación: En caso de que no tengas una respuesta, deja el campo en blanco y/o presiona ENTER.
        ''')

## The Main Function of the Game
This right here is the global function that runs the game, it is composed from all the functions defined in the past cells ...

In [47]:
def basta_ind():
    
    reglas() # starting by showing the rules of the game
    
    run = pyautogui.confirm("Quieres Jugar?", title='BASTA!', buttons = ['Estoy Listo', 'Nah'])
    # this shows a message box in which the player decides to play or not.
    campos = ["Nombre", "Apellido", "Pais", "Fruta", 'Animal', "Color", "Cosa"] # defining the questions or categories
        
    letra = alpha()
    
    path = os.getcwd()+'\\dics'+'\\'+f'{letra}.txt'
    # this should be thoroughly analyzed, as this could be needed to be changed depending on the path format of your computer.

    respuestas = play(run, letra, campos)
    
    if respuestas != None:
        respuestas = list(map(lambda r: '' if r == None else r, respuestas)) # here we replace any None Value answer to a string
        respuestas_limpias = clean_ans(respuestas)
        respuestas_ortografia = respuestas_limpias[:3] + [dictionary(path, respuesta) for respuesta in respuestas_limpias[3:]]
        # here we correct the answers by applying the dictionary function.
        # notice that only the last four categories are going to be corrected, since the first three cannot be found within the                 dictionaries that we downloaded.
        print(respuestas_ortografia, '\n')
        # this is for quality control purposes (a.k.a. debugging).
        respuestas_checadas = check_ans(respuestas_ortografia, letra, campos)
        return finalcard(respuestas_checadas, campos, letra)
    
    else:
        return print('¡Ni Modo Tu Te lo Pierdes!'.upper(), '\n')

In [48]:
basta_ind() # scenario: the player didnt want to play.

¡BASTA!
    
LAS REGLAS DEL JUEGO:

1. Deberás escribir una palabra correspondiente a cada categoría, 
solo tendrás 10 segundos por cada categoría y no cumplir con el tiempo resulta en “timeout”.

2. Cada palabra deberá iniciar con la letra que se escogió al azar.

3. Te ayudaremos con “typos” pero ¡no somos advinos! 

Recomendación: En caso de que no tengas una respuesta, deja el campo en blanco y/o presiona ENTER.
        
¡BU!¡QUE HUEVA ME DAS! 

¡HASTA LA PRÓXIMA! 

¡NI MODO TU TE LO PIERDES! 



In [49]:
basta_ind() # scenario: the player wants to play.

¡BASTA!
    
LAS REGLAS DEL JUEGO:

1. Deberás escribir una palabra correspondiente a cada categoría, 
solo tendrás 10 segundos por cada categoría y no cumplir con el tiempo resulta en “timeout”.

2. Cada palabra deberá iniciar con la letra que se escogió al azar.

3. Te ayudaremos con “typos” pero ¡no somos advinos! 

Recomendación: En caso de que no tengas una respuesta, deja el campo en blanco y/o presiona ENTER.
        
Tus Respuestas Cerebrito:  ['Ge424 rard 99o', 'Escribe lo primero que se te ocurra ...', None, 'g', 'gati', 'griz', 'Timeout'] 

MINUSCULAS: ge424rard99o na na g gati griz na 

PALABRAS: ['gerardo', 'na', 'na', 'g', 'gati', 'griz', 'na'] 

['gerardo', 'na', 'na', 'g', 'gata', 'gris', 'na'] 

Regex Patron ^g[a-z]{2,} 

['gerardo', '???', '???', '???', 'gata', 'gris', '???'] 



,Nombre,Apellido,Pais,Fruta,Animal,Color,Cosa
g,gerardo,???,???,???,gata,gris,???


# To Be Continued ...